# Disclosure

This script is provided purely for educational purposes. 

The following code was based on a script originally by csredino on Github. 
You can find his original code here: https://github.com/csredino/Box-Office-Mojo-Scrapper

And now, a joke at my expense: 
![alt text](http://67.media.tumblr.com/2c9c31d17f47eb1d43fd5ea893360fc7/tumblr_o3nqj5hDzx1rn27pqo1_1280.jpg)

In [4]:
from retrying import retry
from bs4 import BeautifulSoup
from urllib2 import urlopen
import csv
import re
import time
import logging
import httplib
import socket
import pickle

# Defines a function to patch the httplib package
# No, I don't completely understand how/why it works. I shamelessly copied this from Stack Overflow. Only thing I know is this: It works.
def patch_http_response_read(func):
    def inner(*args):
        try:
            return func(*args)
        except httplib.IncompleteRead,e:
            return e.partial
        
    return inner

# Patches httplib package.
httplib.HTTPResponse.read = patch_http_response_read(httplib.HTTPResponse.read)

# Wraps the 'BeautifulSoup' function with a retry component. This is to counteract the occasional internet connection issue. 
# It won't solve all of the problems, but it'll make it so it doesn't have to restart quite so often. 
# That's part of the reason I rewrote the script so that it could be started at different index points. 
@retry()
def BSoup(x):
    print "Souping..." # Prints 'Souping...' to indicate work. Troubleshooting purposes.
    r = BeautifulSoup(x) # Defines a variable to be returned from the function. 
    return(r) # Returns the result of the BeautifulSoup function. 

# Defines an 'alphabet' index, mostly for debugging purposes so I knew when/where the function was failing. 
alphabet= ['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
current_url=("http://www.boxofficemojo.com/movies/alphabetical.htm?letter=NUM&p=.html")# starting point for search. 
movie_links = [] # Initializes movie_links as an empty list. 

soup = BeautifulSoup(urlopen(current_url).read())    # Pulls the content of the 'current_url' to be used for the following function.
letters = soup.findAll('a', href= re.compile('letter=')) # Generate a list of links for the letter indices
letter_index = [] # Initializes 'letter_index' as an empty list. 
for t in letters:
    letter_index.append("http://www.boxofficemojo.com" + t['href']) # Note: This will result in a list of 54 items, essentially listing the pages twice. 
for i in range(0,27): # I added this code to remove duplicates. 
    letter_index.pop()

page_list2 = [] #Declared a second page_list index that wouldn't be reset in the loop. 
for n in range(0,27): # Loop through all letter indices for movies.
    print time.ctime() # Prints the system time. Partially for debugging, partially for indicating that the script is still working.
    print "We are currently scraping alphabet index " + str(alphabet[n]) # Prints index scrape for debugging purposes. 
    current_url = letter_index[n] # First level of segmentation, by letter index. We need to find the second level, by individual page within letter index, to sufficiently compartmentalize data collection.
    print "printing current_url" # For debugging and letting the user know the script is still working. 
    print current_url
    page_list2.append(current_url)
    soup = BSoup(urlopen(current_url).read())
    navbar = soup.find('div', 'alpha-nav-holder')
    pages = navbar.findAll('a', href = re.compile('alphabetical'))
    #print "printing pages..."
    #print pages
    page_list=[] # page_list is apparently reset for each letter. This is why I added the second index.
    
    if pages != None: #this only runs if there is a 2nd page for this letter
        i = 0 # page list starts at 2 (consequence of page layout)
        print len(page_list)
        while i <len(page_list): # loop over multiple pages for each letter index
            #print "printing i"
            #print i
            current_url=page_list[i]
            #print "printing current_url"
            #print current_url
            soup = BSoup(urlopen(current_url).read())
            movietable = soup.find('div',{'id':'main'})
            #print "printing movietable..."
            #print movietable
            movies = movietable.findAll('a',href=re.compile('id='))
            #print "printing movies..."
            #print movies
    #count1 = 1
    for t in pages:
        #print "printing t..."
        #print t
        #print "printing t['href']"
        #print t['href']
        #filename = "pagelist" + str(count1)
        #print "printing page list before"
        #print page_list
        page_list.append("http://www.boxofficemojo.com" + t['href'])
        page_list2.append("http://www.boxofficemojo.com" + t['href'])
        #print "printing page list after"
        #print page_list
        #with open(filename,'w') as f:
         #   pickle.dump(page_list,f)
         #  f.close()
        #count1 = count1+1
    #print "printing page list..."
    #print page_list
    
    movietable = soup.find('div',{'id':'main'})
    movies = movietable.findAll('a', href=re.compile('id='))
    for t in movies:
        movie_links.append("http://www.boxofficemojo.com"+t['href'])
    
    if pages != None: 
        i = 0
        while i<len(page_list):
            current_url = page_list[i]
            print "printing current url"
            print current_url
            soup = BeautifulSoup(urlopen(current_url).read())
            movietable = soup.find('div',{'id':'main'})
            movies = movietable.findAll('a',href=re.compile('id='))
            for t in movies:
                movie_links.append("http://www.boxofficemojo.com" + t['href'])
            i+=1
            #print "printing movies"
            #print movies
            #print "printing movielinks"
            #print movie_links

# I added the following code thinking that I would have to save and iterate through the links each time I wanted to restart.
# That ended up not being the case, so the following code is a bit superfluous. You can comment it out if you'd like.
f = open("movie_links",'w') # Opens a file to save the movie_links list.
pickle.dump(movie_links,f) # Dumps the contents of movie_links into a file.
f.close() # closes the file. 


Sat Nov 05 08:24:30 2016
We are currently scraping alphabet index #
printing current_url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=NUM&p=.htm
Souping...
0
Sat Nov 05 08:24:31 2016
We are currently scraping alphabet index a
printing current_url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&p=.htm
Souping...
0
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=2&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=3&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=4&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=5&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=6&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=7&p=.htm
printing current url
http://www.boxofficemojo.com/movies/alphabetical.htm?

In [5]:
print movie_links

['http://www.boxofficemojo.com/movies/?id=horrorifc.htm', 'http://www.boxofficemojo.com/movies/?id=9dot99.htm', 'http://www.boxofficemojo.com/movies/?id=supercapitalist.htm', 'http://www.boxofficemojo.com/movies/?id=500daysofsummer.htm', 'http://www.boxofficemojo.com/movies/?id=untitled.htm', 'http://www.boxofficemojo.com/movies/?id=andjusticeforall.htm', 'http://www.boxofficemojo.com/movies/?id=1mileabove.htm', 'http://www.boxofficemojo.com/movies/?id=1plus1.htm', 'http://www.boxofficemojo.com/movies/?id=1000rupeenote.htm', 'http://www.boxofficemojo.com/movies/?id=1000times.htm', 'http://www.boxofficemojo.com/movies/?id=10.htm', 'http://www.boxofficemojo.com/movies/?id=badrobot2016.htm', 'http://www.boxofficemojo.com/movies/?id=10daysinamadhouse.htm', 'http://www.boxofficemojo.com/movies/?id=10itemsorless.htm', 'http://www.boxofficemojo.com/movies/?id=10questionsforthedalailama.htm', 'http://www.boxofficemojo.com/movies/?id=10rules.htm', 'http://www.boxofficemojo.com/movies/?id=10thin

In [16]:
# I rewrote the original script so that the following code was executed as an individual function instead of a looped script. 
# I was then able to write a loop that would pass the individual url and opened file for writing the scraped data. 
# Because of this, I could more easily see when an issue came up and then start it again without having to restart the whole process. 




def getmoviedata(url,open_file):
    # The following block counteracts an indexing issue for two of the urls.
    if "elizabeth" in url and "elizabethtown" not in url:
        url = 'http://www.boxofficemojo.com/movies/?id=elizabeth%A0.htm'
    if "simpleplan" in url:
        url = 'http://www.boxofficemojo.com/movies/?id=simpleplan%A0.htm'
    
    # The following code prints the link and time being scanned so that the user knows where to look if issues arise. 
    print time.ctime()
    print url
    current_url = (url + "&adjust_yr=2015&p=.htm") 
    soup = BSoup(urlopen(current_url).read())
    directors = soup.findAll('a',href=re.compile('Director&id'))
    director_list=[]
    for t in directors:
        director_list.append(t.encode_contents())
    for i in range(0,2):
        if i>=len(director_list):
            director_list.append('N/A')#fill rest of list
    director1=director_list[0]
    director2=director_list[1]

    writers=soup.findAll('a', href= re.compile('Writer&id'))
    writer_list=[]
    for t in writers:
        writer_list.append(t.encode_contents())
    for i in range(0,2):
        if i>=len(writer_list):
            writer_list.append('N/A')
    writer1=writer_list[0]
    writer2=writer_list[1]

    composers=soup.findAll('a', href= re.compile('Composer&id'))
    composer_list=[]
    for t in composers:
        composer_list.append(t.encode_contents())
    for i in range(0,2):
        if i>=len(composer_list):
            composer_list.append('N/A')
    composer1=composer_list[0]
    composer2=composer_list[1]

    actors=soup.findAll('a', href= re.compile('Actor&id'))
    actor_list=[]
    for t in actors:
        actor_list.append(t.encode_contents())
    for i in range(0,6):
        if i>=len(actor_list):
            actor_list.append('N/A')
    actor1=actor_list[0]
    actor2=actor_list[1]
    actor3=actor_list[2]
    actor4=actor_list[3]
    actor5=actor_list[4]
    actor6=actor_list[5]

    producers=soup.findAll('a', href= re.compile('Producer&id'))
    producer_list=[]
    for t in producers:
        producer_list.append(t.encode_contents())
    for i in range(0,6):
        if i>=len(producer_list):
            producer_list.append('N/A')
    producer1=producer_list[0]
    producer2=producer_list[1]
    producer3=producer_list[2]
    producer4=producer_list[3]
    producer5=producer_list[4]
    producer6=producer_list[5]


    all_bs=soup.findAll('b')
    b_list=[] #lots of the information we want is in bold, and appears in the same order on each page
    for t in all_bs:
        #print t
        if 'Domestic Lifetime' not in t.encode_contents():#want to ignore the lifetime box office
            b_list.append(t.encode_contents())
    if len(b_list)>=10:#avoids bad entries with no box office data
        if '$'in b_list[2] or 'n/a' in b_list[9]:#avoid movies w/o box office data, or unadjustable box office data, if not caught above
            if 'n/a' in b_list[9]:#has a foreign release only, order is shifted
                title=b_list[1]
                domestic='N/A'
                if 'N/A' not in b_list[2]:
                    distributor=b_list[2].split('>')[1].split('<')[0]
                else:
                    distributor=b_list[2]
                if len(b_list[3].split('>'))>3:#sometimes the release date is not in a hyperlink
                    release=b_list[3].split('>')[2].split('<')[0]
                else:
                    release=b_list[3].split('>')[1].split('<')[0]
                genre=b_list[4]
                runtime=b_list[5]
                rating=b_list[6]
                budget=b_list[7]
                worldwide=b_list[12]
            else:	#has a domestic release
                title=b_list[1]
                domestic=b_list[2]
                if 'n/a' not in b_list[3]:
                    distributor=b_list[3].split('>')[1].split('<')[0]
                else:
                    distributor=b_list[3]
                if len(b_list[4].split('>'))>3:#sometimes the release date is not in a hyperlink
                    release=b_list[4].split('>')[2].split('<')[0]
                else:
                    release=b_list[4].split('>')[1].split('<')[0]
                genre=b_list[5]
                runtime=b_list[6]
                rating=b_list[7]
                budget=b_list[8]
                if len(b_list)==11 or '%' not in b_list[11]:#this means it only has a domestic release
                    worldwide='N/A'
                else:
                    worldwide=b_list[13]
            #print release
            print time.ctime()
            print "writing..."
            open_file.writerow([title,director1,director2,domestic,distributor,release,genre,runtime,rating,budget,worldwide,actor1,actor2,actor3,actor4,actor5,actor6,producer1,producer2,producer3,producer4,producer5,producer6,writer1,writer2,composer1,composer2])#since this is in the big "if" it wont write to file if it is formated incorrectly
            
            



In [15]:
f.close()
print "file closed. Safe to delete."

file closed. Safe to delete.


In [17]:
# The following code opens a csv file and prepares it for receiving the scraped data. 
f = open("moviedump.csv",'wb')
fieldnames = ("title","director1","director2","domestic", "distributor","release","genre","runtime","rating","budget","worldwide","actor1","actor2","actor3","actor4","actor5","actor6","producer1","producer2","producer3","producer4","producer5","producer6","writer1","writer2","composer1","composer2")
output = csv.writer(f, delimiter=",")
output.writerow(fieldnames)

# I set up the following code so that the csv file wouldn't be closed unless the scrape process was complete. 
# In the case of an error, the script will return a message indicating at what index point the script failed. 
# The user can then continue the process by executing the script in the following cell. 
progress = 0
completion = len(movie_links)
Executed = False
Error = False
while Executed == False:
    # The following 'if' conditionals are designed to skip bad links I have been unable to rectify. 
    # It's possible that these pages are just temporarily disabled or malfunctioning. 
    if progress == 1478:
        progress+=1
    if progress == 3313:
        progress+=1
    if progress == 5410:
        progress+=1
    if progress == 8728:
        progress+=1
    if progress == 8730:
        progress +=1
    if progress == 10921:
        progress +=1
    try:
        getmoviedata(movie_links[progress],output)
    except:
        Executed = True
        Error = True
    
    if Executed == False:
        progress+=1
    if progress >= completion:
        Executed = True


if Error == False:
    f.close()
    print "Data Scraped Successfully"

else:
    errorpoint = progress
    print "Data scrape interrupted at index ", str(errorpoint)


Sat Nov 05 08:31:32 2016
http://www.boxofficemojo.com/movies/?id=horrorifc.htm
Souping...
Sat Nov 05 08:31:32 2016
http://www.boxofficemojo.com/movies/?id=9dot99.htm
Souping...
Sat Nov 05 08:31:32 2016
writing...
Sat Nov 05 08:31:32 2016
http://www.boxofficemojo.com/movies/?id=supercapitalist.htm
Souping...
Sat Nov 05 08:31:33 2016
writing...
Sat Nov 05 08:31:33 2016
http://www.boxofficemojo.com/movies/?id=500daysofsummer.htm
Souping...
Sat Nov 05 08:31:33 2016
writing...
Sat Nov 05 08:31:33 2016
http://www.boxofficemojo.com/movies/?id=untitled.htm
Souping...
Sat Nov 05 08:31:34 2016
writing...
Sat Nov 05 08:31:34 2016
http://www.boxofficemojo.com/movies/?id=andjusticeforall.htm
Souping...
Sat Nov 05 08:31:34 2016
writing...
Sat Nov 05 08:31:34 2016
http://www.boxofficemojo.com/movies/?id=1mileabove.htm
Souping...
Sat Nov 05 08:31:34 2016
http://www.boxofficemojo.com/movies/?id=1plus1.htm
Souping...
Sat Nov 05 08:31:34 2016
writing...
Sat Nov 05 08:31:34 2016
http://www.boxofficemojo.c

In [28]:
# Since the 'progress' variable will not be changed in the case of an error, the user can just run the current cell without any changes.

Executed = False
Error = False
while Executed == False:
    if progress == 1478:
        progress+=1
    if progress == 1490:
        progress+=1
    if progress == 3313:
        progress+=1
    if progress == 3335:
        progress+=1
    if progress == 4127:
        progress+=1
    if progress == 5410:
        progress+=1
    if progress == 5440:
        progress+=1
    if progress == 8728:
        progress+=1
    if progress == 8782:
        progress+=1
    if progress == 8784:
        progress+=1
    if progress == 8730:
        progress +=1
    if progress == 10357:
        progress +=1
    if progress == 10921:
        progress +=1
    if progress == 14798:
        progress +=1
    try:
        getmoviedata(movie_links[progress],output)
    except:
        Executed = True
        Error = True
    
    if Executed == False:
        progress+=1
    if progress >= completion:
        Executed = True


if Error == False:
    f.close()
    print "Data Scraped Successfully"

else:
    errorpoint = progress
    print "Data scrape interrupted at index ", str(errorpoint)


Sat Nov 05 21:25:48 2016
http://www.boxofficemojo.com/movies/?id=thingstodoindenverwhenyouredead.htm
Souping...
Sat Nov 05 21:25:48 2016
writing...
Sat Nov 05 21:25:48 2016
http://www.boxofficemojo.com/movies/?id=thingswelostinthefire.htm
Souping...
Sat Nov 05 21:25:49 2016
writing...
Sat Nov 05 21:25:49 2016
http://www.boxofficemojo.com/movies/?id=thinklikeaman.htm
Souping...
Sat Nov 05 21:25:49 2016
writing...
Sat Nov 05 21:25:49 2016
http://www.boxofficemojo.com/movies/?id=thinklikeaman2.htm
Souping...
Sat Nov 05 21:25:49 2016
writing...
Sat Nov 05 21:25:49 2016
http://www.boxofficemojo.com/movies/?id=thirdman.htm
Souping...
Sat Nov 05 21:25:50 2016
http://www.boxofficemojo.com/movies/?id=thethirdman15.htm
Souping...
Sat Nov 05 21:25:50 2016
writing...
Sat Nov 05 21:25:50 2016
http://www.boxofficemojo.com/movies/?id=thirdman99.htm
Souping...
Sat Nov 05 21:25:50 2016
writing...
Sat Nov 05 21:25:50 2016
http://www.boxofficemojo.com/movies/?id=thirdmiracle.htm
Souping...
Sat Nov 05 21:

In [27]:
print(len(movie_links))


16789
